In [1]:
import os

In [2]:
%pwd

'/Users/sameerraj/Desktop/Machine_learning_python/End-to-End-ML-red-wine-prediction-project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/sameerraj/Desktop/Machine_learning_python/End-to-End-ML-red-wine-prediction-project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_namr: Path
    target_column: str

In [6]:
from src.mlproject.constant import *
from src.mlproject.utils.common import read_yaml,create_directories

In [7]:
class ConfigrationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_namr=config.metric_file_name,
            target_column=schema.name
        )
        return model_evaluation_config


In [8]:
import os 
import pandas as pd   
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import numpy as np
import joblib
import json

In [9]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config=config

    def eval_metrics(self,actual,pred):
        rmse=np.sqrt(mean_absolute_error(actual,pred))
        mae=mean_absolute_error(actual,pred)
        r2=r2_score(actual,pred)
        return r2_score,mae 
    
    def save_results(self):
        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        x_test=test_data.drop([self.config.target_column])
        y_test=test_data[[self.config.target_column]]

        predicted_qualities=model.predict(x_test)

        (rmse,mae,r2)=self.eval_metrics(y_test,predicted_qualities)

        # Saving metrics as local
        scores = {"rmse": rmse, "mae": mae, "r2": r2}
        json.save_json(path=Path(self.config.metric_file_name), data=scores)

In [10]:
try:
    config=ConfigrationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation_config=ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2023-10-23 12:51:21,515: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-23 12:51:21,516: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-23 12:51:21,519: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-23 12:51:21,519: INFO: common: created directory at : artifacts]
[2023-10-23 12:51:21,520: INFO: common: created directory at : artifacts/model_evaluation]


KeyError: "['quality'] not found in axis"